In [12]:
# ============================================
# 0) Setup básico: Montar Drive + librerías
# ============================================
!pip install -U numpy==1.26.4 pandas==2.2.2 matplotlib==3.9.0 opencv-python-headless==4.10.0.84 scikit-learn==1.5.1 tensorflow==2.16.1 --quiet

import os, random, cv2, json, math, gc, itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from google.colab import drive
drive.mount('/content/driveNawa')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.9/589.9 MB 845.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
tensorstore 0.1.78 requires ml_dtypes>=0.5.0, but you have ml-dtypes 0.3.2 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.16.1 which is incompatible.
jax 0.5.3 requires ml_dtypes>=0.4.0, but you have ml-dtypes 0.3.2 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is inc

In [13]:

# ============================================
# 1) Rutas maestras (ajusta DRIVE_ROOT si hace falta)
# ============================================
DRIVE_ROOT = "/content/driveNawa/MyDrive/proyecto_conjuntiva"  # <- cambia si tu carpeta tiene otro nombre
CSV_PATH   = f"{DRIVE_ROOT}/metadata.csv"                  # CSV: filepath (relativo a IMG_ROOT), patient_id, hb, label
IMG_ROOT   = f"{DRIVE_ROOT}/images"                        # carpeta con imágenes

os.makedirs(f"{DRIVE_ROOT}/models", exist_ok=True)
os.makedirs(f"{DRIVE_ROOT}/logs", exist_ok=True)

In [14]:
# ============================================
# 2) Cargar metadatos y validar columnas
# ============================================
df = pd.read_csv(CSV_PATH)

# Normalizamos nombres esperados
expected_cols = {"filepath", "patient_id", "hb", "label"}
missing = expected_cols - set(df.columns.str.lower())
if missing:
    raise ValueError(f"El CSV debe tener columnas {expected_cols}. Faltan: {missing}")

# Estándar: aseguramos tipos
df.columns = [c.lower() for c in df.columns]
df["filepath"]   = df["filepath"].astype(str)
df["patient_id"] = df["patient_id"].astype(str)
df["label"]      = df["label"].astype(int)

# Chequeo rápido de existencia de archivos
def exists_row(row):
    return os.path.exists(os.path.join(IMG_ROOT, row["filepath"]))
df["exists"] = df.apply(exists_row, axis=1)
if not df["exists"].all():
    faltantes = df.loc[~df["exists"], "filepath"].head(20).tolist()
    raise FileNotFoundError(f"Hay imágenes faltantes (mostrando hasta 20): {faltantes}")

print(f"Total filas: {len(df)} | Pacientes únicos: {df['patient_id'].nunique()} | Positivos (1): {df['label'].sum()}")

Total filas: 26 | Pacientes únicos: 26 | Positivos (1): 13


In [15]:
# ============================================
# 3) Split por paciente (GroupShuffleSplit)
# ============================================
RANDOM_SEED = 123
gss = GroupShuffleSplit(n_splits=1, train_size=0.6, random_state=RANDOM_SEED)
train_idx, temp_idx = next(gss.split(df, groups=df["patient_id"]))
df_train = df.iloc[train_idx].reset_index(drop=True)
df_temp  = df.iloc[temp_idx].reset_index(drop=True)

# Validación y test 15%/15% por paciente
gss2 = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=RANDOM_SEED)
val_idx, test_idx = next(gss2.split(df_temp, groups=df_temp["patient_id"]))
df_val  = df_temp.iloc[val_idx].reset_index(drop=True)
df_test = df_temp.iloc[test_idx].reset_index(drop=True)

print("Split por paciente ->",
      f"train: {len(df_train)} ({df_train['patient_id'].nunique()} pacientes)",
      f"val:   {len(df_val)} ({df_val['patient_id'].nunique()} pacientes)",
      f"test:  {len(df_test)} ({df_test['patient_id'].nunique()} pacientes)", sep="\n")

Split por paciente ->
train: 15 (15 pacientes)
val:   5 (5 pacientes)
test:  6 (6 pacientes)


In [16]:
# ============================================
# 4) Preprocesado de imagen (solo imágenes)
#    - Gray-world white balance
#    - CLAHE en canal V (HSV)
# ============================================
IMG_SIZE = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE

def gray_world_balance_np(bgr):
    bgr = bgr.astype(np.float32)
    avgB, avgG, avgR = bgr.mean(axis=(0,1))
    avgGray = (avgB + avgG + avgR) / 3.0
    bgr[:,:,0] *= (avgGray / (avgB + 1e-6))
    bgr[:,:,1] *= (avgGray / (avgG + 1e-6))
    bgr[:,:,2] *= (avgGray / (avgR + 1e-6))
    return np.clip(bgr, 0, 255).astype(np.uint8)

def clahe_v_np(bgr):
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    v = hsv[:,:,2]
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    hsv[:,:,2] = clahe.apply(v)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def load_and_preprocess_np(path):
    # path: bytes -> str
    path = path.decode("utf-8")
    bgr = cv2.imread(path, cv2.IMREAD_COLOR)
    if bgr is None:
        raise ValueError(f"No pude leer la imagen: {path}")
    # Preprocesado
    bgr = gray_world_balance_np(bgr)
    bgr = clahe_v_np(bgr)
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    rgb = cv2.resize(rgb, IMG_SIZE, interpolation=cv2.INTER_AREA)
    return rgb

def tf_preprocess(path, label):
    # path ya es absoluto; tf.numpy_function devuelve uint8
    img = tf.numpy_function(load_and_preprocess_np, [path], tf.uint8)
    img = tf.ensure_shape(img, (*IMG_SIZE, 3))
    img = tf.cast(img, tf.float32) / 255.0
    return img, tf.cast(label, tf.float32)

In [17]:
# ============================================
# 5) Construir tf.data pipelines
# ============================================
def build_ds(frame, batch=32, shuffle=False, augment=False):
    abs_paths = frame["filepath"].apply(lambda p: os.path.join(IMG_ROOT, p)).tolist()
    labels    = frame["label"].tolist()
    ds = tf.data.Dataset.from_tensor_slices((abs_paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(frame), seed=RANDOM_SEED, reshuffle_each_iteration=True)
    ds = ds.map(lambda p, y: (tf.convert_to_tensor(p), y))
    ds = ds.map(tf_preprocess, num_parallel_calls=AUTOTUNE)

    if augment:
        aug = keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.05),
            layers.RandomContrast(0.1),
            layers.RandomBrightness(0.1),
        ])
        ds = ds.map(lambda x, y: (aug(x, training=True), y), num_parallel_calls=AUTOTUNE)

    ds = ds.batch(batch).prefetch(AUTOTUNE)
    return ds

BATCH = 32
train_ds = build_ds(df_train, batch=BATCH, shuffle=True, augment=True)
val_ds   = build_ds(df_val,   batch=BATCH, shuffle=False, augment=False)
test_ds  = build_ds(df_test,  batch=BATCH, shuffle=False, augment=False)

In [18]:
# ============================================
# 6) Modelo: EfficientNetB0 (warmup + fine-tune)
# ============================================
base = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=(*IMG_SIZE, 3), pooling="avg")
base.trainable = False

inputs = keras.Input(shape=(*IMG_SIZE, 3))
x = base(inputs, training=False)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

# Class weights por desbalance
pos = df_train["label"].sum()
neg = len(df_train) - pos
class_weight = {0: 1.0, 1: (neg / (pos + 1e-6))}
print("class_weight:", class_weight)

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[keras.metrics.AUC(name="AUC"), keras.metrics.Precision(name="Precision"), keras.metrics.Recall(name="Recall")]
)

ckpt_path = f"{DRIVE_ROOT}/models/best_warmup.keras"
cbs = [
    keras.callbacks.ModelCheckpoint(ckpt_path, monitor="val_AUC", mode="max", save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor="val_AUC", mode="max", patience=6, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_AUC", mode="max", patience=3, factor=0.3, verbose=1)
]

hist = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=cbs,
    class_weight=class_weight
)

# Fine-tune: desbloquea últimas ~50 capas
base.trainable = True
for layer in base.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(3e-5),
    loss="binary_crossentropy",
    metrics=[keras.metrics.AUC(name="AUC"), keras.metrics.Precision(name="Precision"), keras.metrics.Recall(name="Recall")]
)

ckpt_path_ft = f"{DRIVE_ROOT}/models/best_finetune.keras"
cbs_ft = [
    keras.callbacks.ModelCheckpoint(ckpt_path_ft, monitor="val_AUC", mode="max", save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor="val_AUC", mode="max", patience=6, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_AUC", mode="max", patience=3, factor=0.3, verbose=1)
]

hist_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=cbs_ft,
    class_weight=class_weight
)
assert os.path.exists(CSV_PATH), f"CSV no encontrado: {CSV_PATH}"
assert os.path.isdir(IMG_ROOT),  f"Carpeta de imágenes no encontrada: {IMG_ROOT}"

class_weight: {0: 1.0, 1: 1.4999997500000417}
Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33s/step - AUC: 0.4444 - Precision: 0.3750 - Recall: 0.5000 - loss: 0.8440
Epoch 1: val_AUC improved from -inf to 0.58333, saving model to /content/driveNawa/MyDrive/proyecto_conjuntiva/models/best_warmup.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 39s 39s/step - AUC: 0.4444 - Precision: 0.3750 - Recall: 0.5000 - loss: 0.8440 - val_AUC: 0.5833 - val_Precision: 0.6000 - val_Recall: 1.0000 - val_loss: 0.6839 - learning_rate: 0.0010
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - AUC: 0.6296 - Precision: 0.4167 - Recall: 0.8333 - loss: 0.8155
Epoch 2: val_AUC did not improve from 0.58333
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - AUC: 0.6296 - Precision: 0.4167 - Recall: 0.8333 - loss: 0.8155 - val_AUC: 0.5000 - val_Precision: 0.6000 - val_Recall: 1.0000 - val_loss: 0.6857 - learning_rate: 0.0010
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - AUC: 0.2407 - Precision: 0.2727 - Recall: 0.5000 - loss: 0.8879
Epoch 3: val_A

In [19]:
from sklearn.metrics import (
    roc_auc_score, roc_curve,
    confusion_matrix, classification_report,
    precision_score
)

# ============================================
# 7) Evaluación en TEST + umbral por sensibilidad
# ============================================
# Recolectar probabilidades y etiquetas
y_true, y_prob = [], []
for xb, yb in test_ds:
    p = model.predict(xb, verbose=0).ravel()
    y_prob.extend(p.tolist())
    y_true.extend(yb.numpy().tolist())

y_true = np.array(y_true).astype(int)
y_prob = np.array(y_prob).astype(float)

# Calcular AUC y curva ROC
auc = roc_auc_score(y_true, y_prob)
fpr, tpr, thr = roc_curve(y_true, y_prob)

# Elegir umbral que logra ~90% sensibilidad (si existe)
target_sens = 0.90
idx = np.argmax(tpr >= target_sens)
thr_sens = thr[idx] if (idx < len(thr)) else 0.5
print(f"AUC (test): {auc:.4f} | Umbral @≈{target_sens*100:.0f}% Sens: {thr_sens:.4f}")

# Predicciones binarias con el umbral elegido
y_pred = (y_prob >= 0.5).astype(int) # Ajustar el umbral
# y_pred = (y_prob >= thr_sens).astype(int)

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
print("Matriz de confusión @thr_sens:\n", cm)

# Precisión
precision = precision_score(y_true, y_pred)
print(f"Precisión: {precision:.4f}")

# Reporte de clasificación
print("\nReporte de clasificación:\n", classification_report(y_true, y_pred, digits=4))

AUC (test): 0.7500 | Umbral @≈90% Sens: 0.5272
Matriz de confusión @thr_sens:
 [[0 2]
 [0 4]]
Precisión: 0.6667

Reporte de clasificación:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         2
           1     0.6667    1.0000    0.8000         4

    accuracy                         0.6667         6
   macro avg     0.3333    0.5000    0.4000         6
weighted avg     0.4444    0.6667    0.5333         6



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
import tensorflow as tf

# Guarda el modelo Keras primero
final_keras = f"{DRIVE_ROOT}/models/conjuntiva_final.keras"
model.save(final_keras)
print("Modelo guardado:", final_keras)

# Conversión a TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Si hay un problema de firma, asegúrate de que la entrada sea la correcta
converter.input_shapes = {model.input.name: [None, *IMG_SIZE, 3]}  # Ajusta el tamaño de entrada

# Convertir
tflite_model = converter.convert()

# Guardar el modelo convertido
tflite_path = f"{DRIVE_ROOT}/models/conjuntiva_float32.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print("TFLite guardado:", tflite_path)

Modelo guardado: /content/driveNawa/MyDrive/proyecto_conjuntiva/models/conjuntiva_final.keras
Saved artifact at '/tmp/tmpxpwab9ob'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_492')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  138747545367760: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  138747545368528: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  138748124598480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138748127046800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138748127047184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138748124598864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138748124596752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138748127049104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138748127049296: TensorSpec(shape=(), 

In [ ]:
# ============================================
# 9) Función de inferencia para una sola imagen
# ============================================
def predict_one(abs_img_path, thr=thr_sens):
    # Devuelve probabilidad y clase con el preprocesado usado en entrenamiento
    rgb = load_and_preprocess_np(abs_img_path)  # devuelve RGB 224x224
    x = rgb.astype(np.float32) / 255.0
    p = float(model.predict(x[None,...], verbose=0)[0,0])
    return p, int(p >= thr)

# Ejemplo de uso:
# test_path = os.path.join(IMG_ROOT, df_test.iloc[0]["filepath"])
# prob, cls = predict_one(test_path)
# print("Prob(pálido):", prob, "Predicción:", cls)